### Sprint 2: Model Development

In [63]:
# Feature Engineering for Water Pump Classification
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [72]:
data_path = os.path.join(os.getcwd(), '..', 'data')
out_path = os.path.join(os.getcwd(), '..', 'outputs')     # For the output

# Read the files
train = pd.read_csv(os.path.join(data_path, 'cleaned_data_filled_V6.csv'))
labels = pd.read_csv(os.path.join(data_path, 'train_labels.csv'))

# Merge training labels
df = pd.merge(train, labels, on='id')
print('Files successfully loaded and merged!')


Files successfully loaded and merged!


## Ticket 2.1.1: New Features 

In [80]:
# Age of pump
current_year = 2025
df['pump_age'] = current_year - df['construction_year']

# Manual vs. motorized extraction
df['is_manual_pump'] = df['extraction_type_class'].str.lower().str.contains('hand').astype(int)

# water_risk_score

# 1. Water Quality + Quantity Combination: Create 'water_risk_score'
# Map water_quality into numeric risk levels
water_quality_map = {
    'soft': 1, 'good': 1,
    'salty': 2, 'milky': 2,
    'coloured': 3, 'fluoride': 3, 'fluoride, salty': 4, 'salty abandoned': 4
}
df['water_quality_numeric'] = df['water_quality'].map(water_quality_map)

# Combine amount_tsh_capped and water quality
df['water_risk_score'] = df['amount_tsh_capped'] * df['water_quality_numeric']

## Ticket 2.1.2: Transform existing features to improve model performance 

In [86]:
# First we want to see distribution of all numerical values, Select only numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

#  Drop ID and target columns
numerical_cols = [col for col in numerical_cols if col not in ['id', 'status_group_numeric']]

#Calculate skewness
from scipy.stats import skew
skewness_results = {}

for col in numerical_cols:
    skew_val = skew(df[col].dropna())
    skewness_results[col] = skew_val

# Sort by skewness
skewness_sorted = dict(sorted(skewness_results.items(), key=lambda item: abs(item[1]), reverse=True))

# Show results
print("Feature Skewness:")
for feature, skew_val in skewness_sorted.items():
    print(f"{feature}: {skew_val:.3f}")




Feature Skewness:
num_private: 90.521
amount_tsh: 56.938
water_risk_score: 8.917
has_private_owner: 8.549
amount_tsh_capped: 8.062
district_code: 3.902
population: 3.699
water_quality_numeric: 3.483
region_code: 3.142
amount_tsh_log: 1.290
amount_tsh_capped_log: 1.281
population_log: -1.278
water_risk_score_log: 1.238
pump_age: 1.232
construction_year: -1.232
is_manual_pump: 1.073
gps_height: -0.612
latitude: -0.252
recorded_year: -0.145
longitude: -0.135


#### Decision Based on Skewness Analysis
Apply log1p transformation (log(x+1)) to the following features to reduce skewness:num_private, amount_tsh, water_risk_score, amount_tsh_capped ,population, pump_age

#### For other features, no transformation is necessary at this stage.

#### New features will be saved with a _log suffix (e.g., amount_tsh_log) to preserve original columns.


In [97]:
# List of skewed columns for log-transform
skewed_cols = ['amount_tsh', 'water_risk_score', 'amount_tsh_capped', 'population' ]

# Apply log1p transformation and create new columns with _log suffix
for col in skewed_cols:
    df[col + '_log'] = np.log1p(df[col])

In [102]:
#check the skewness after log-transforming
for col in skewed_cols:
    print(f"{col}_log skewness: {skew(df[col + '_log'].dropna()):.3f}")

amount_tsh_log skewness: 1.290
water_risk_score_log skewness: 1.238
amount_tsh_capped_log skewness: 1.281
population_log skewness: -1.278


#### num_private is extremely sparse: most pumps have no private owner (value = 0).
#### Only a tiny number of pumps have some private ownership information.
#### It's not a continuous variable in reality — it behaves like a binary feature (yes/no private ownership). 0 → No private owner 1 → Private owner exists

In [105]:
df['has_private_owner'] = (df['num_private'] > 0).astype(int)


In [108]:
# Bin pump_age into categories
df['pump_age_binned'] = pd.cut(
    df['pump_age'],
    bins=[0, 10, 30, 100],
    labels=['young', 'medium', 'old']
)

In [110]:
df.to_csv(os.path.join(data_path,"feature_engineered_data_V1.csv"), index=False)
print('Data saved successfully: feature_engineered_data_V1.csv')

Data saved successfully: feature_engineered_data_V1.csv
